In [1]:
import numpy as np
import collections
import itertools
from sklearn.ensemble import RandomForestClassifier
import re

# Day 4: Passport Processing
You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

```
byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
cid (Country ID)
```

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

```
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
```

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [2]:
def hot_encode(keys):
    # ['byr', 'cid', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid']
    he = np.array([0, 0, 0, 0, 0, 0, 0, 0])
    if 'byr' in keys:
        he += [1, 0, 0, 0, 0, 0, 0, 0]
    if 'cid' in keys:
        he += [0, 1, 0, 0, 0, 0, 0, 0]
    if 'ecl' in keys:
        he += [0, 0, 1, 0, 0, 0, 0, 0]
    if 'eyr' in keys:
        he += [0, 0, 0, 1, 0, 0, 0, 0]
    if 'hcl' in keys:
        he += [0, 0, 0, 0, 1, 0, 0, 0]
    if 'hgt' in keys:
        he += [0, 0, 0, 0, 0, 1, 0, 0]
    if 'iyr' in keys:
        he += [0, 0, 0, 0, 0, 0, 1, 0]
    if 'pid' in keys:
        he += [0, 0, 0, 0, 0, 0, 0, 1]
    return he

def read_credentials_from_file(filname, apply_hot_encoding=False):
    creds = []
    with open(filname, 'r') as f:
        cred = {}
        for line in f:
            if line == '\n':
                if apply_hot_encoding:
                    creds.append(hot_encode(dict(sorted(cred.items())).keys()))
                else:
                    creds.append(dict(sorted(cred.items())))
                cred = {}
                continue
            
            kv = [vals.split(':') for vals in line.strip().split(' ')]
            for key, value in kv:
                cred[key] = value

        # save last
        if apply_hot_encoding:
            creds.append(hot_encode(dict(sorted(cred.items())).keys()))
        else:
            creds.append(dict(sorted(cred.items())))
    return creds

In [3]:
# prepare training data

X_train = np.array(list(itertools.product([0, 1], repeat=8)))
y_train = np.array([False] * X_train.shape[0])

idx_1 = np.all(X_train == [1, 1, 1, 1, 1, 1, 1, 1], axis=1)
y_train[idx_1] = True
idx_2 = np.all(X_train == [1, 0, 1, 1, 1, 1, 1, 1], axis=1)
y_train[idx_2] = True

In [4]:
clf = RandomForestClassifier(n_estimators=1, bootstrap=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, n_estimators=1)

In [5]:
prediction = clf.predict(read_credentials_from_file('input/d4.dat', apply_hot_encoding=True))
prediction.sum()

228

# Part Two
The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

```
byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
```

Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:

```
byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
```

Here are some invalid passports:

```
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
```

Here are some valid passports:

```
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
```

Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

In [6]:
creds = read_credentials_from_file('input/d4.dat')
creds[0:1]

[{'byr': '1976',
  'eyr': '2024',
  'hcl': '#fffffd',
  'hgt': '187cm',
  'iyr': '2016',
  'pid': '827837505'}]

```
byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
```

In [7]:
rules = {
    'byr': {
        'min': 1920,
        'max': 2002,
    },
    'iyr': {
        'min': 2010,
        'max': 2020,
    },
    'eyr': {
        'min': 2020,
        'max': 2030,
    },
    'hgt': {
        'shape': r'\d+(in|cm)',
        'min': {
            'in': 59,
            'cm': 150,
        },
        'max': {
            'in': 76,
            'cm': 193,
        },
    },
    'hcl': {
        'shape': r'#[0-9a-f]{6}',
    },
    'ecl': {
        'one_of': ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']
    },
    'pid': {
        'shape': r'\d{9}',
    },
    'cid': {}
}

rules

{'byr': {'min': 1920, 'max': 2002},
 'iyr': {'min': 2010, 'max': 2020},
 'eyr': {'min': 2020, 'max': 2030},
 'hgt': {'shape': '\\d+(in|cm)',
  'min': {'in': 59, 'cm': 150},
  'max': {'in': 76, 'cm': 193}},
 'hcl': {'shape': '#[0-9a-f]{6}'},
 'ecl': {'one_of': ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']},
 'pid': {'shape': '\\d{9}'},
 'cid': {}}

In [8]:
def verify(creds, rules):
    prediction = clf.predict([hot_encode(creds.keys())])
    print('creds', creds)
    print('\tprediction', prediction)
    if not prediction:
        return False

    for key in creds.keys():
        print(f'\trules[{key}]', rules[key])
        for rule in rules[key].keys():
            print('\t\trule', rule)
            if rule == 'min':
                m = rules[key]['min']
                if type(m) is int:
                    if int(creds[key]) < m:
                        return False
                if type(m) is dict:
                    print('\t\tmin', m)
                    for min_key in m.keys():
                        if min_key in creds[key]:
                            val = int(creds[key].strip(min_key))
                            if val < m[min_key]:
                                return False
            if rule == 'max':
                m = rules[key]['max']
                if type(m) is int:
                    if int(creds[key]) > m:
                        return False
                if type(m) is dict:
                    print('\t\tmax', m)
                    for max_key in m.keys():
                        if max_key in creds[key]:
                            val = int(creds[key].strip(max_key))
                            if val > m[max_key]:
                                return False
            if rule == 'shape':
                m = rules[key]['shape']
                print('\t\tshape', creds[key], re.fullmatch(m, creds[key]))
                if re.fullmatch(m, creds[key]) is None:
                    return False
            if rule == 'one_of':
                if creds[key] not in rules[key]['one_of']:
                    return False
                
    return True

In [9]:
np.array([verify(creds, rules) for creds in read_credentials_from_file('input/d4.dat')]).sum()

	rule max
	rules[hcl] {'shape': '#[0-9a-f]{6}'}
		rule shape
		shape #a97842 <re.Match object; span=(0, 7), match='#a97842'>
	rules[hgt] {'shape': '\\d+(in|cm)', 'min': {'in': 59, 'cm': 150}, 'max': {'in': 76, 'cm': 193}}
		rule shape
		shape 186cm <re.Match object; span=(0, 5), match='186cm'>
		rule min
		min {'in': 59, 'cm': 150}
		rule max
		max {'in': 76, 'cm': 193}
	rules[iyr] {'min': 2010, 'max': 2020}
		rule min
		rule max
	rules[pid] {'shape': '\\d{9}'}
		rule shape
		shape 033811215 <re.Match object; span=(0, 9), match='033811215'>
creds {'byr': '1972', 'ecl': 'oth', 'eyr': '2029', 'hcl': '#a97842', 'hgt': '160cm', 'iyr': '2020', 'pid': '535511110'}
	prediction [ True]
	rules[byr] {'min': 1920, 'max': 2002}
		rule min
		rule max
	rules[ecl] {'one_of': ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']}
		rule one_of
	rules[eyr] {'min': 2020, 'max': 2030}
		rule min
		rule max
	rules[hcl] {'shape': '#[0-9a-f]{6}'}
		rule shape
		shape #a97842 <re.Match object; span=(0, 7), match

175